This predictor uses "Indoor Navigation and Location Wifi Features" data by [@hiro5299834](https://www.kaggle.com/hiro5299834), which is based on data by [@devinanzelmo](https://www.kaggle.com/devinanzelmo).

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, glob
from catboost import CatBoostClassifier, CatBoostRegressor

In [ ]:
test_df = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv')
test_df.head()

In [ ]:
feature_dir = "../input/indoor-navigation-and-location-wifi-features"
train_files = sorted(glob.glob(os.path.join(feature_dir, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_dir, '*_test.csv')))
test_df['site'] = test_df['site_path_timestamp'].apply(lambda x: x.split('_')[0])

In [ ]:
for i, site in enumerate(test_df['site'].unique()):
    train_data = pd.read_csv(train_files[i], index_col=0)
    test_data = pd.read_csv(test_files[i], index_col=0)
    X_train = train_data.drop(['x', 'y', 'f', 'path'], axis=1)
    X_test = test_data.drop(['site_path_timestamp'], axis=1)

    f_model = CatBoostClassifier(verbose=False)
    f_model.fit(X_train, train_data['f'])
    test_df.loc[test_df['site']==site, 'floor'] = f_model.predict(X_test)

    model = CatBoostRegressor(verbose=False, objective='RMSE', depth=10)

    model.fit(X_train, train_data['x'])
    test_df.loc[test_df['site']==site, 'x'] = model.predict(X_test)

    model.fit(X_train, train_data['y'])
    test_df.loc[test_df['site']==site, 'y'] = model.predict(X_test)

In [ ]:
test_df.drop(['site'], axis=1).to_csv('submission.csv', index=None)
test_df.head()